### Yolo Mini Project
비눗방울 객체탐지

1차적으로 데이터를 수집하여 annotation을 1차적으로 진행했으며, 수집한 방식은 아래와 같다.
- 구글에서 약 50장 가량의 데이터 수집하고, 다이소에서 구매한 버블건을 활용하여 실제 사진을 찍기도 함
- LabelImg를 활용한 어노테이션 진행
- 72장의 train images, 21개의 validation images, 17개의 test images 수집 완료

사전에 훈련을 진행해보았을 때, 너무 작은 비눗방울의 경우 제대로된 탐지를 하지 못하는 문제점이 있었다.

즉시 어노테이션 파일을 점검한 결과로 어노테이션을 수정하고, 좀 더 타이트하게 어노테이션을 진행했다. 

<div align=center><img src="./Wrong Annotation.png"></div>

이후 학습을 진행했을 때 작은 물체까지 잘 잡아내는 모습을 보였으나, 이에 대한 정확도를 높이기 위해 추가적으로 데이터를 수집한다.

### 필요 라이브러리 import

In [2]:
import os
import cv2
import yaml
import json
import torch
import shutil
from crawler.pix_crawling import PixCrawler  # 직접 만든 클래스
from imutils.video import WebcamVideoStream

### Image Url 수집
기존의 이미지와 픽사베이에서 수집한 이미지를 병합한다.

In [4]:
users_input = ["Nature background"]  # 여러개를 검색하려면 요소 추가할 것
# keywords = users_input.split(", ")

for key in users_input:
    if key.startswith(" "):
        key.replace(key[0], "")
    else:
        pass

my_crawler = PixCrawler(
    keywords=users_input,
    num_of_data=10,
    path_datasets="./Append",
    path_output_datasets="./output",
)
my_crawler.dowload_datasets()

Nature background 데이터 다운로드를 시작합니다.


percentage: 100%|=====================| 66/66 [00:05<00:00, 11.09it/s]

Nature background 66개 중 66개 이미지를 다운받았습니다. 
이미지 다운로드가 완료되었습니다. ./Append폴더를 확인해주세요. 


### 이미지 1차 전처리
이미 있는 데이터들이거나, 상관없는 이미지들의 경우 삭제한다. 

### 이미지 이동
176장 중 불필요한 데이터를 제거하고 총 34장의 데이터를 확보했다. 이전 결과와 똑같이 비교하기 위해 28장은 훈련셋 이미지로, 6장은 검증셋 이미지로 사용하며 테스트셋에는 별도로 더 추가하지 않는다.

In [4]:
image_dir = "./Append/Bubble"
image_list = os.listdir(image_dir)

for img in image_list[:28]:
    shutil.move(f"{image_dir}/{img}", f"./BubbleData/train/images/{img}")

for img2 in image_list[28:]:
    shutil.move(f"{image_dir}/{img2}", f"./BubbleData/validation/images/{img2}")

### 추가 데이터에 대한 라벨링 작업
추가적인 데이터에 대한 라벨링 작업을 진행한다.

In [ ]:
%cd C:\Users\OWNER\Desktop\labeling\labelImg-master
!python labelimg.py

In [5]:
%cd D:\Intel\YoloProject

D:\Intel\YoloProject


### clone yolov5

In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


### Yaml 파일 수정

In [3]:
with open("./yolov5/data/coco.yaml", "r") as f:
    data = yaml.safe_load(f)

data["path"] = r"D:\Intel\YoloProject\BubbleData"
data["train"] = "train"
data["test"] = "test"
data["val"] = "validation"

data["nc"] = 2
data["names"] = ["bubble", "background"]

with open("./yolov5/data/coco.yaml", "w") as file:
    yaml.dump(data, file)

In [9]:
%pwd

'D:\\Intel\\YoloProject'

### 훈련
clone한 파일들 중 `train.py`를 이용해서 훈련을 진행한다. argparse로 작성되어 여러 옵션들을 함께 지정해주어야 한다.
- --batch : batch_size
- --epochs : 훈련 횟수
- --data : data정보를 담고있는 yaml 파일
- --weights : pretrained 된 가중치를 적용할 때 사용. 지정하지 않으면 랜덤한 가중치와 편향값들로 학습 진행. 해당 파일이 존재하지 않을 경우 다운로드를 받고 사용한다.
- --cfg : 모델을 담고있는 yaml파일. s ➡️ n ➡️ m ➡️ l ➡️ x 순으로 복잡도가 다르다. 

In [ ]:
!python ./yolov5/train.py --batch 30 --epochs 100 --data "D:\Intel\YoloProject\yolov5\data\Car.yaml"  --weights ./yolov5/yolov5n.pt --cfg ./yolov5/models/yolov5n.yaml

### 사진 검출
위에서 훈련한 나만의 모델을 기반으로 새로운 이미지들에 대한 객체 탐지 결과를 `detect.py`파일로 받아본다. 마찬가지로 argparse로 작성되어있다.
- --source : 탐지할 이미지 경로
- --weights : 나의 학습 모델의 가중치 경로
- --conf : 객체 탐지 신뢰도. 해당 신뢰도 이상의 객체들만 탐지한다. 기본값은 0.4이며 높을 수록 탐지되는 객체의 수도 적어진다.
- --name : 탐지된 파일들을 저장할 경로

In [ ]:
!python ./yolov5/detect.py --source "D:\Intel\YoloProject\BubbleData\test\images" --weights "./yolov5/runs/train/exp/weights/best.pt" --conf 0.65 --name "C:\Users\OWNER\Desktop\bubble_detection_test\results"

### Droid Cam or 동영상을 활용한 객체 탐지
Custom data의 pt파일을 적용시켜 Droid Cam을 활용해 검출하거나, 동영상에서 검출한다.

### 모델 정의

In [3]:
model = torch.hub.load(
    r"D:\Intel\YoloProject\yolov5",
    "custom",
    path=r"D:\Intel\YoloProject\yolov5\runs\train\exp11\weights\best.pt",
    source="local",
)
model.conf = 0.4
# model.classes = [0]

YOLOv5  v7.0-214-g8c30c58 Python-3.9.12 torch-1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 


### 바운딩 박스 정보 반환 함수 정의
바운딩 박스에 대한 좌표값을 json형태로 return하는 함수를 정의한다.

In [4]:
def get_attributes(frame):
    results = model(frame)
    results.render()

    return json.loads(results.pandas().xyxy[0].to_json(orient="records"))

# def allocate_ids(frame):
#     bounded_boxes = get_attributes(frame)
    
#     if len(bounded_boxes) != 0:
#         for i, dit in enumerate(bounded_boxes):
#             dit['id'] = i
    
#     return bounded_boxes

### 검출
Droid Cam을 활용하거나, Video를 활용하여 검출한다.

In [ ]:
esp_ip = "http://192.168.0.39"
host = "{}:4747/video".format(esp_ip)
cam = WebcamVideoStream(src=host).start()
# cam = cv2.VideoCapture(r"C:\Users\OWNER\Desktop\video_file.mp4")

while True:
    frame = cam.read() # ret,
    # if ret:
    ano_box = get_attributes(frame)
    cv2.imshow("frame", frame)
    # for dit in ano_box:
    #     print(dit['id'])
    # if 1 < len(ano_box) < 5:
    #     print("위험")
    # elif 6 < len(ano_box) < 10:
    #     print("고위험")
    # else:
    #     print("작별인사")

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    # else:
    #     break